In [10]:
import pandas as pd

# Read data
data = pd.read_excel('data.xlsx', sheet_name='PS19A_PERSON__SmplR10K', keep_default_na=False)
#data = pd.read_csv('data_small.tsv.win.utf8.txt', sep='\t', keep_default_na=False)

# Extract addresses
addresses = data['person_address']
del data

addressCount = len(addresses)

data = {}

In [11]:
import Levenshtein as lev
import multiprocessing
import threading
import math

# Number of threads to create
threadCount = multiprocessing.cpu_count()

# Amount of addresses to use per thread
threadAddressCount = int(math.ceil(float(addressCount) / threadCount))

class CalculateDistanceThread(threading.Thread):
    def __init__(self, threadID):
        threading.Thread.__init__(self)
        self.threadID = threadID
    def run(self):
        start = threadAddressCount * self.threadID
        for n in range(threadAddressCount):
            if (start + n < addressCount):
                for x in range(addressCount):
                    ratio = lev.ratio(addresses[start + n].lower(), addresses[x].lower())
                    if (ratio > 0.6):
                        data[(n, x)] = ratio

# Create n threads
for n in range(threadCount):
    t = CalculateDistanceThread(n)
    t.start()
    t.join()

del threadCount

In [12]:
from pprint import pprint

pprint(list(data.items())[:8])

[((0, 0), 1.0),
 ((1, 1), 1.0),
 ((1, 131), 0.6015037593984962),
 ((1, 531), 0.7972972972972973),
 ((1, 849), 0.7853881278538813),
 ((1, 879), 0.7809523809523811),
 ((2, 2), 1.0),
 ((3, 3), 1.0)]
